In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
RANDOM_STATE = 110

train_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/train_data.csv")
test_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/test_data.csv")

In [3]:
def plot_box(dataframe, column_name):
    """
    주어진 데이터프레임과 열 이름에 대해 박스 플롯을 그리는 함수.

    Parameters:
    dataframe (pd.DataFrame): 데이터프레임
    column_name (str): 열 이름
    """
    plt.figure(figsize=(10, 6))
    plt.boxplot(dataframe[column_name], vert=False)
    plt.xlabel(column_name)
    plt.title(f'Box Plot of {column_name}')
    plt.show()
    
def value_counts_ratio_count(df, col_name, target_name):
    """
    주어진 데이터프레임의 특정 열에 대해 각 값마다 타겟 변수의 비율과 갯수, 총 갯수를 출력하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    target_name (str): 타겟 변수 이름
    """
    # 각 값마다 타겟 변수의 비율 계산
    value_counts = df.groupby(col_name)[target_name].value_counts(normalize=True).unstack().fillna(0)
    
    # 각 값마다 타겟 변수의 갯수 계산
    counts = df.groupby(col_name)[target_name].value_counts().unstack().fillna(0)
    
    # 각 값마다 총 갯수 계산
    total_counts = df[col_name].value_counts().rename('Total_Count')
    
    # 비율과 갯수를 합침
    result = value_counts.join(counts, lsuffix='_ratio', rsuffix='_count')
    
    # 총 갯수를 합침
    result = result.join(total_counts, on=col_name)
    
    # 출력 형식 조정
    result.index.name = 'variable'
    print(f"\n{col_name}별 {target_name} 비율 및 갯수\n")
    print(result.rename(columns=lambda x: x.split('_')[0]))
    

def summarize_grouped_data(df, group_by_columns):
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_columns)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # 'target' 변수의 'AdNormal' 비율과 갯수 계산
        adnormal_count = group['target'].value_counts().get('AbNormal', 0)
        adnormal_ratio = adnormal_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, adnormal_count, adnormal_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', "'AdNormal' count", 'ratio', 'Total'])
    
    # 그룹화된 변수들의 이름을 제목행으로 출력
    print(f"Grouped by: {', '.join(group_by_columns)}")
    print()
    # 데이터프레임 출력
    print(results_df)
    
    return results_df
  
  
def plot_abnormal_ratio(dataframe, column_name, target_name, target_value, bins=20):
    """
    주어진 데이터프레임의 특정 열에 대해 각 값마다 타겟 변수의 특정 값 비율을 계산하고 막대그래프로 표시하는 함수.

    Parameters:
    dataframe (pd.DataFrame): 데이터프레임
    column_name (str): 열 이름
    target_name (str): 타겟 변수 이름
    target_value (str): 타겟 변수의 특정 값
    bins (int): 구간의 수 (기본값은 20)
    """
    def abnormal_ratio(dataframe, column_name, target_name, target_value):
        """
        주어진 데이터프레임의 특정 열에 대해 각 값마다 타겟 변수의 특정 값 비율을 계산하는 함수.

        Parameters:
        dataframe (pd.DataFrame): 데이터프레임
        column_name (str): 열 이름
        target_name (str): 타겟 변수 이름
        target_value (str): 타겟 변수의 특정 값

        Returns:
        pd.DataFrame: 각 값마다 타겟 변수의 특정 값 비율을 포함하는 데이터프레임
        """
        # 각 값마다 타겟 변수의 특정 값 비율 계산
        value_counts = dataframe.groupby(column_name)[target_name].apply(lambda x: (x == target_value).mean()).reset_index()
        count_counts = dataframe.groupby(column_name)[target_name].count().reset_index()
        
        value_counts.columns = [column_name, 'ratio']
        count_counts.columns = [column_name, 'count']
        
        # 비율과 카운트를 병합
        result = pd.merge(value_counts, count_counts, on=column_name)
        return result

    # column_name 값을 지정된 구간으로 나누기
    dataframe[f'{column_name}_bins'] = pd.cut(dataframe[column_name], bins=bins)

    # 비율 계산
    ratios = abnormal_ratio(dataframe, f'{column_name}_bins', target_name, target_value)

    # 막대그래프 그리기
    plt.figure(figsize=(20, 10))
    barplot = sns.barplot(x=f'{column_name}_bins', y='ratio', data=ratios, color='skyblue')
    plt.xlabel(f'{column_name} (binned)')
    plt.ylabel('AbNormal Ratio')
    plt.title(f'AbNormal Ratio by {column_name} (binned)', pad=30)  # 제목과 그래프 사이의 간격 조정
    plt.xticks(rotation=45)
    plt.ylim(0, 1)

    # 각 막대 위에 비율 값과 카운트 표시
    for p in barplot.patches:
        # 막대의 x 좌표에 해당하는 구간을 찾기
        bin_label = ratios[f'{column_name}_bins'].cat.categories[int(p.get_x() + p.get_width() / 2) - 1]
        count_value = ratios.loc[ratios[f'{column_name}_bins'] == bin_label, 'count'].values[0]
        barplot.annotate(f'{format(p.get_height(), ".2f")} ({count_value})', 
                         (p.get_x() + p.get_width() / 2., p.get_height()), 
                         ha='center', va='center', 
                         xytext=(0, 9), 
                         textcoords='offset points')

    plt.show()

    # _bins 변수 드랍
    dataframe.drop(columns=[f'{column_name}_bins'], inplace=True)
    
    
# train_data와 test_data에서 '?'를 포함하는 열 이름 필터링
train_Process_Desc_col = train_data.filter(like='?').columns
test_Process_Desc_col = test_data.filter(like='?').columns

# 필터링된 열 이름 출력
print("<? column in train_data>")
for col in train_Process_Desc_col:
    print(col)

print("<? column in test_data>")
for col in test_Process_Desc_col:
    print(col)

# ? -> Θ로 변경할 열 이름과 새 열 이름 생성
train_new_columns = {col: col.replace('?', 'Θ') for col in train_Process_Desc_col}
test_new_columns = {col: col.replace('?', 'Θ') for col in test_Process_Desc_col}

# 열 이름 변경
train_data.rename(columns=train_new_columns, inplace=True)
test_data.rename(columns=test_new_columns, inplace=True)

# 'Θ'를 포함하는 열 이름 필터링
train_Process_Desc_col = train_data.filter(like='Θ').columns
test_Process_Desc_col = test_data.filter(like='Θ').columns

# 필터링된 열 이름 출력
print("<Θ in train_data>")
print("train_data:")
for col in train_Process_Desc_col:
    print(col)

print("test_data:")
for col in test_Process_Desc_col:
    print(col)
    
# '_Dam'를 포함하는 열 이름 필터링
Process_Desc_col = train_data.filter(like='_Dam').columns

# 필터링된 열 이름 출력
print("<Dam 공정 관련 변수>")
for col in Process_Desc_col:
    print(col)

<? column in train_data>
CURE END POSITION ? Collect Result_Dam
CURE START POSITION ? Collect Result_Dam
<? column in test_data>
<Θ in train_data>
train_data:
CURE END POSITION Θ Collect Result_Dam
CURE START POSITION Θ Collect Result_Dam
test_data:
CURE END POSITION Θ Collect Result_Dam
CURE START POSITION Θ Collect Result_Dam
<Dam 공정 관련 변수>
CURE END POSITION X Collect Result_Dam
CURE END POSITION Z Collect Result_Dam
CURE END POSITION Θ Collect Result_Dam
CURE SPEED Collect Result_Dam
CURE START POSITION X Collect Result_Dam
CURE START POSITION Θ Collect Result_Dam
DISCHARGED SPEED OF RESIN Collect Result_Dam
DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam
DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam
DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam
Dispense Volume(Stage1) Collect Result_Dam
Dispense Volume(Stage2) Collect Result_Dam
Dispense Volume(Stage3) Collect Result_Dam
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam
HEAD NORMAL COORDINATE X AXIS(Stage1) J

HEAD Clean position Z 분석

In [4]:
train_data['Head Clean Position Z Collect Result_Dam'].value_counts()

Head Clean Position Z Collect Result_Dam
130.85    23418
124.00     8493
133.50     8231
124.50      331
118.85       33
Name: count, dtype: int64

In [5]:
summary_df = summarize_grouped_data(train_data, ['Model.Suffix','Head Clean Position Z Collect Result_Dam'])

Grouped by: Model.Suffix, Head Clean Position Z Collect Result_Dam

                    group  'AdNormal' count     ratio  Total
0   (AJX75334501, 118.85)                 0  0.000000     33
1    (AJX75334501, 124.0)               200  0.041459   4824
2    (AJX75334501, 124.5)                28  0.084592    331
3   (AJX75334501, 130.85)               965  0.047184  20452
4    (AJX75334501, 133.5)               725  0.088631   8180
5    (AJX75334502, 124.0)                58  0.068884    842
6   (AJX75334502, 130.85)               163  0.065226   2499
7    (AJX75334502, 133.5)                 9  0.183673     49
8    (AJX75334503, 124.0)                 1  0.010417     96
9   (AJX75334503, 130.85)                43  0.671875     64
10   (AJX75334503, 133.5)                 0  0.000000      2
11   (AJX75334505, 124.0)               128  0.050354   2542
12  (AJX75334505, 130.85)                 2  0.021505     93
13   (AJX75334506, 124.0)                 2  0.083333     24
14  (AJX75334506,

In [6]:
# 파생변수 생성: 3개의 컬럼 값이 모두 동일하면 해당 값을 저장, 아니면 diff
train_data['Receip_No'] = train_data.apply(
    lambda row: row['Receip No Collect Result_Dam'] if (row['Receip No Collect Result_Dam'] == row['Receip No Collect Result_Fill1'] == row['Receip No Collect Result_Fill2']) else 'diff',
    axis=1
)
test_data['Receip_No'] = test_data.apply(
    lambda row: row['Receip No Collect Result_Dam'] if (row['Receip No Collect Result_Dam'] == row['Receip No Collect Result_Fill1'] == row['Receip No Collect Result_Fill2']) else 'diff',
    axis=1
)

In [7]:
# 파생변수 생성: Receip No와 Model.Suffix의 조합
train_data['model_receip'] = train_data['Model.Suffix'] + '_' + train_data['Receip_No'].astype(str)
test_data['model_receip'] = test_data['Model.Suffix'] + '_' + test_data['Receip_No'].astype(str)

# 파생변수 생성: workorder 앞 4자리 -> workorder_prefix
train_data['workorder_prefix'] = train_data['Workorder'].str[:4]
test_data['workorder_prefix'] = test_data['Workorder'].str[:4]

# 파생변수 생성: Receip No와 workorder_prefix의 조합 -> diff, 3.0, 9.0의 경우에만
train_data['workorder_receip'] = train_data.apply(
    lambda row: f"{row['workorder_prefix']}_{row['Receip_No']}" 
    if row['Receip_No'] in ['diff', 3.0, 9.0] else row['workorder_prefix'],
    axis=1
)
test_data['workorder_receip'] = test_data.apply(
    lambda row: f"{row['workorder_prefix']}_{row['Receip_No']}" 
    if row['Receip_No'] in ['diff', 3.0, 9.0] else row['workorder_prefix'],
    axis=1
)

In [9]:
train_data['workorder_receip'].value_counts()

workorder_receip
3G1X         4608
3M1X         4585
3J1X         4057
4A1X         3962
3L1X         3665
3H1X         3322
3K1X         3176
4B1X         2852
4E1X         2696
3I1X         2227
4C1X         2103
3F1X         1276
4F1X          960
3LPM          153
4BPM          139
3KPM          123
3JPX          100
3MPX           78
4BPX_9         68
3HPX_3         64
3HPM           63
4CPM           53
4EPM           46
3FPM           24
3HPX_9         24
3GPM           23
3HPX           21
3KPX           11
4APX            9
4CPX            8
4APX_9          4
4B1X_diff       3
3FPX            2
3M1X_diff       1
Name: count, dtype: int64

In [11]:
summary_df = summarize_grouped_data(train_data, ['workorder_prefix','Head Clean Position Z Collect Result_Dam'])

Grouped by: workorder_prefix, Head Clean Position Z Collect Result_Dam

             group  'AdNormal' count     ratio  Total
0    (3F1X, 124.5)                28  0.084592    331
1    (3F1X, 133.5)                85  0.089947    945
2    (3FPM, 133.5)                 0  0.000000     24
3    (3FPX, 133.5)                 0  0.000000      2
4    (3G1X, 133.5)               435  0.094401   4608
5    (3GPM, 133.5)                 1  0.043478     23
6   (3H1X, 130.85)                60  0.079365    756
7    (3H1X, 133.5)               209  0.081450   2566
8    (3HPM, 133.5)                 4  0.063492     63
9    (3HPX, 124.0)                 0  0.000000     24
10  (3HPX, 130.85)                46  0.541176     85
11  (3I1X, 130.85)               165  0.074091   2227
12  (3J1X, 130.85)               190  0.046833   4057
13  (3JPX, 130.85)                 4  0.040000    100
14  (3K1X, 130.85)               100  0.031486   3176
15  (3KPM, 130.85)                 6  0.048780    123
16  (3KPX,

In [10]:
summary_df = summarize_grouped_data(train_data, ['workorder_receip','Head Clean Position Z Collect Result_Dam'])

Grouped by: workorder_receip, Head Clean Position Z Collect Result_Dam

                  group  'AdNormal' count     ratio  Total
0         (3F1X, 124.5)                28  0.084592    331
1         (3F1X, 133.5)                85  0.089947    945
2         (3FPM, 133.5)                 0  0.000000     24
3         (3FPX, 133.5)                 0  0.000000      2
4         (3G1X, 133.5)               435  0.094401   4608
5         (3GPM, 133.5)                 1  0.043478     23
6        (3H1X, 130.85)                60  0.079365    756
7         (3H1X, 133.5)               209  0.081450   2566
8         (3HPM, 133.5)                 4  0.063492     63
9        (3HPX, 130.85)                 3  0.142857     21
10     (3HPX_3, 130.85)                43  0.671875     64
11      (3HPX_9, 124.0)                 0  0.000000     24
12       (3I1X, 130.85)               165  0.074091   2227
13       (3J1X, 130.85)               190  0.046833   4057
14       (3JPX, 130.85)                 4  

Head Purge Position Z 변수

In [14]:
summary_df = summarize_grouped_data(train_data, ['workorder_receip',
                                                 'Head Clean Position Z Collect Result_Dam'
                                                 , 'Head Purge Position Z Collect Result_Dam'
                                                 ])

Grouped by: workorder_receip, Head Clean Position Z Collect Result_Dam, Head Purge Position Z Collect Result_Dam

                          group  'AdNormal' count     ratio  Total
0          (3F1X, 124.5, 124.5)                28  0.084592    331
1          (3F1X, 133.5, 133.5)                85  0.089947    945
2          (3FPM, 133.5, 133.5)                 0  0.000000     24
3          (3FPX, 133.5, 133.5)                 0  0.000000      2
4          (3G1X, 133.5, 133.5)               435  0.094401   4608
5          (3GPM, 133.5, 133.5)                 1  0.043478     23
6        (3H1X, 130.85, 130.85)                58  0.078273    741
7         (3H1X, 130.85, 133.5)                 2  0.133333     15
8          (3H1X, 133.5, 133.5)               209  0.081450   2566
9          (3HPM, 133.5, 133.5)                 4  0.063492     63
10       (3HPX, 130.85, 130.85)                 3  0.142857     21
11     (3HPX_3, 130.85, 130.85)                43  0.671875     64
12      (3HPX_9

Head Zero Position Y 분석

In [12]:
summary_df = summarize_grouped_data(train_data, ['workorder_receip','Head Zero Position Y Collect Result_Dam'])

Grouped by: workorder_receip, Head Zero Position Y Collect Result_Dam

                 group  'AdNormal' count     ratio  Total
0        (3F1X, 303.5)               113  0.088558   1276
1        (3FPM, 303.5)                 0  0.000000     24
2        (3FPX, 303.5)                 0  0.000000      2
3        (3G1X, 300.0)               110  0.088710   1240
4        (3G1X, 303.5)               325  0.096496   3368
5        (3GPM, 303.5)                 1  0.043478     23
6        (3H1X, 300.0)               269  0.080975   3322
7        (3HPM, 300.0)                 4  0.063492     63
8        (3HPX, 300.0)                 3  0.142857     21
9      (3HPX_3, 303.5)                43  0.671875     64
10     (3HPX_9, 300.0)                 0  0.000000     24
11       (3I1X, 300.0)               165  0.074091   2227
12       (3J1X, 300.0)               190  0.046833   4057
13       (3JPX, 300.0)                 4  0.040000    100
14       (3K1X, 300.0)               100  0.031486   3176
1

In [13]:
summary_df = summarize_grouped_data(train_data, ['workorder_receip',
                                                 'Head Clean Position Z Collect Result_Dam'
                                                 , 'Head Zero Position Y Collect Result_Dam'
                                                 ])

Grouped by: workorder_receip, Head Clean Position Z Collect Result_Dam, Head Zero Position Y Collect Result_Dam

                         group  'AdNormal' count     ratio  Total
0         (3F1X, 124.5, 303.5)                28  0.084592    331
1         (3F1X, 133.5, 303.5)                85  0.089947    945
2         (3FPM, 133.5, 303.5)                 0  0.000000     24
3         (3FPX, 133.5, 303.5)                 0  0.000000      2
4         (3G1X, 133.5, 300.0)               110  0.088710   1240
5         (3G1X, 133.5, 303.5)               325  0.096496   3368
6         (3GPM, 133.5, 303.5)                 1  0.043478     23
7        (3H1X, 130.85, 300.0)                60  0.079365    756
8         (3H1X, 133.5, 300.0)               209  0.081450   2566
9         (3HPM, 133.5, 300.0)                 4  0.063492     63
10       (3HPX, 130.85, 300.0)                 3  0.142857     21
11     (3HPX_3, 130.85, 303.5)                43  0.671875     64
12      (3HPX_9, 124.0, 300.0

Thickness 분석

In [16]:
summary_df = summarize_grouped_data(train_data, ['THICKNESS 1 Collect Result_Dam','THICKNESS 2 Collect Result_Dam'])

Grouped by: THICKNESS 1 Collect Result_Dam, THICKNESS 2 Collect Result_Dam

              group  'AdNormal' count     ratio  Total
0  (-0.054, -0.219)                37  0.041855    884
1  (-0.019, -0.021)                74  0.044876   1649
2  (-0.015, -0.036)                72  0.074689    964
3        (0.0, 0.0)              2091  0.059036  35419
4   (0.012, -0.022)                 2  0.016000    125
5   (0.014, -0.058)                62  0.051796   1197
6    (0.014, 0.007)                 6  0.040000    150
7    (0.037, 0.005)                 6  0.050847    118


In [17]:
summary_df = summarize_grouped_data(train_data, ['Model.Suffix',
                                                 'THICKNESS 1 Collect Result_Dam'
                                                 , 'THICKNESS 2 Collect Result_Dam'
                                                 ])

Grouped by: Model.Suffix, THICKNESS 1 Collect Result_Dam, THICKNESS 2 Collect Result_Dam

                            group  'AdNormal' count     ratio  Total
0   (AJX75334501, -0.054, -0.219)                17  0.029463    577
1   (AJX75334501, -0.019, -0.021)                37  0.039828    929
2   (AJX75334501, -0.015, -0.036)                35  0.081967    427
3         (AJX75334501, 0.0, 0.0)              1810  0.057776  31328
4    (AJX75334501, 0.012, -0.022)                 2  0.016000    125
5    (AJX75334501, 0.014, -0.058)                13  0.040373    322
6     (AJX75334501, 0.014, 0.007)                 4  0.045977     87
7     (AJX75334501, 0.037, 0.005)                 0  0.000000     25
8   (AJX75334502, -0.054, -0.219)                20  0.071685    279
9   (AJX75334502, -0.019, -0.021)                13  0.057269    227
10  (AJX75334502, -0.015, -0.036)                12  0.120000    100
11        (AJX75334502, 0.0, 0.0)               177  0.064954   2725
12   (AJX7533

In [15]:
summary_df = summarize_grouped_data(train_data, ['workorder_receip',
                                                 'THICKNESS 1 Collect Result_Dam'
                                                 , 'THICKNESS 2 Collect Result_Dam'
                                                 ])

Grouped by: workorder_receip, THICKNESS 1 Collect Result_Dam, THICKNESS 2 Collect Result_Dam

                       group  'AdNormal' count     ratio  Total
0           (3F1X, 0.0, 0.0)               113  0.088558   1276
1           (3FPM, 0.0, 0.0)                 0  0.000000     24
2           (3FPX, 0.0, 0.0)                 0  0.000000      2
3           (3G1X, 0.0, 0.0)               435  0.094401   4608
4           (3GPM, 0.0, 0.0)                 1  0.043478     23
5           (3H1X, 0.0, 0.0)               269  0.080975   3322
6           (3HPM, 0.0, 0.0)                 4  0.063492     63
7           (3HPX, 0.0, 0.0)                 3  0.142857     21
8         (3HPX_3, 0.0, 0.0)                43  0.671875     64
9   (3HPX_9, -0.054, -0.219)                 0  0.000000     24
10          (3I1X, 0.0, 0.0)               165  0.074091   2227
11          (3J1X, 0.0, 0.0)               190  0.046833   4057
12          (3JPX, 0.0, 0.0)                 4  0.040000    100
13        